In [ ]:
import rasterio
import affine
from rasterio import plot
import numpy as np
import shapefile
from math import sqrt
import os
from decimal import *
os.environ['CUDA_VISIBLE_DEVICES']='1'
np.seterr(divide='ignore', invalid='ignore') #Il se peut qu'il y ait des divisions par 0, numpy retourne 0 dans ces cas.

In [ ]:
IMAGE_PIXEL = 30
DEFAULT = "not_labeled"
S1DIRECTORY = "output/sentinel1v2"
S1NAME = "tile_"

In [ ]:
bands = ['../Données/S1Spring/measurement/s1a-iw-grd-vv-20200408t223649-20200408t223714-032042-03b3ce-001.tiff',
         '../Données/S1Spring/measurement/s1a-iw-grd-vh-20200408t223649-20200408t223714-032042-03b3ce-002.tiff',
         '../Données/S1Summer/measurement/s1a-iw-grd-vv-20200713t223654-20200713t223719-033442-03e00d-001.tiff',
         '../Données/S1Summer/measurement/s1a-iw-grd-vh-20200713t223654-20200713t223719-033442-03e00d-002.tiff',
         '../Données/S1Fall/measurement/s1a-iw-grd-vv-20191011t223652-20191011t223717-029417-035883-001.tiff',
         '../Données/S1Fall/measurement/s1a-iw-grd-vh-20191011t223652-20191011t223717-029417-035883-002.tiff',
         '../Données/S1Winter/measurement/s1a-iw-grd-vv-20200115t223649-20200115t223714-030817-038911-001.tiff',
         '../Données/S1Winter/measurement/s1a-iw-grd-vh-20200115t223649-20200115t223714-030817-038911-002.tiff'
        ]
bands_for_blue = ['../Données/S1Spring/measurement/s1a-iw-grd-vh-20200408t223649-20200408t223714-032042-03b3ce-002.tiff',
                  '../Données/S1Summer/measurement/s1a-iw-grd-vh-20200713t223654-20200713t223719-033442-03e00d-002.tiff',
                  '../Données/S1Fall/measurement/s1a-iw-grd-vh-20191011t223652-20191011t223717-029417-035883-002.tiff',
                  '../Données/S1Winter/measurement/s1a-iw-grd-vh-20200115t223649-20200115t223714-030817-038911-002.tiff']

In [ ]:
import shapefile
import math

In [ ]:
LIST_SHAPEFILES = ["../Données/heigvd/central_highlands_2_other/central_highlands_2_other.shp",
                  "../Données/heigvd/central_highlands_2_test/central_highlands_2_test.shp",
                  "../Données/heigvd/central_higlands_1_other/central_higlands_1_other.shp"]
CORRESPONDANCES = {"1":"cacao", "2":"coffee","3":"complex_oil", "4":"nativevege", "5":"oil_palm", "6":"rubber", "7":"unknown", 
                 "8":"seasonal", "9":"urban", "10":"water", "11":"other_tree", "12":"other_no_tree", "13":"native_no_tree",
                 "14":"water_other", "15":"pepper", "16":"cassava", "17":"tea", "18":"rice", "19":"banana", "20":"baby_palm", 
                 "21":"cur_off-regrow", "22":"natural_wetland", "23":"intercrop", "24":"deciduous_forest", "25":"stick_pepper", 
                 "26":"flooded_plantation", "27":"pine_trees", "28":"coconut", "29":"banboo", "30":"savana", "31":"mango", 
                 "32":"other_fruit_tree_crop", "33":"water_mine", "0":"not_labeled", "-1":"ambiguous"}

In [ ]:
points = []
for path in LIST_SHAPEFILES:
    sf = shapefile.Reader(path)
    shapes = sf.shapes()
    for point in sf.records():
        points.append((shapes[point.oid].points[0], point[0]))

In [ ]:
import SentinelFonction as sf
def get_corners(gcps):
    #Calcul des gcps voisins
    gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left = None, None, None, None
    for gcp in gcps:
        if gcp.row <= 0 and gcp.col <= 0:
            gcp_top_right = gcp
        elif gcp.row <= 0 and gcp.col >= IMAGE_PIXEL and (gcp_top_left == None or gcp_top_left.col >= gcp.col):
            gcp_top_left = gcp
        elif gcp.row >= IMAGE_PIXEL and gcp.col <= 0 and (gcp_bottom_right == None or gcp_bottom_right.row >= gcp.row):
            gcp_bottom_right = gcp
        elif gcp.row >= IMAGE_PIXEL and gcp.col >= IMAGE_PIXEL:
            gcp_bottom_left = gcp
            break
        
    #Pour les cas en bordure
    if gcp_bottom_left == None:
        gcp_bottom_left = gcps[-1]
    if gcp_bottom_right == None or gcp_top_left == None:
        for gcp in gcps:
            if gcp.row == gcp_top_right.row and gcp.col == gcp_bottom_left.col:
                gcp_top_left = gcp
            elif gcp.row == gcp_bottom_left.row and gcp.col == gcp_top_right.col:
                gcp_bottom_right = gcp
            if gcp_bottom_right != None and gcp_top_left != None:
                break
    #Calcul des vecteurs
    v1, v2 = sf.get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left)
    #Calcul des coins
    top_right = (np.array([Decimal(gcp_top_right.x), Decimal(gcp_top_right.y)]) - 
                 Decimal(gcp_top_right.col) * v1) - Decimal(gcp_top_right.row) * v2
    top_left = top_right + Decimal(IMAGE_PIXEL) * v1
    bottom_right = top_right + Decimal(IMAGE_PIXEL) * v2
    bottom_left = bottom_right + Decimal(IMAGE_PIXEL) * v1
    return {"tr":top_right, "tl":top_left, "br":bottom_right, "bl":bottom_left}

In [ ]:
def translate_gcps(gcps, image_pixel=30, x=0, y=0):
    result = []
    for gcp in gcps:
        result.append(rasterio.control.GroundControlPoint(row=gcp.row - x * image_pixel, col=gcp.col - y * image_pixel, 
                                                          x=gcp.x, y=gcp.y, id=gcp.id))
    return result

In [ ]:
import sys
from sympy import symbols, Eq, solve
def get_labels(gcps):
    """Donne tous les labels contenu dans une image de Sentinel-1"""
    corners = get_corners(gcps)
    labels = set()
    v1, v2 = sf.get_simple_vectors(corners["tr"], corners["tl"], corners["br"], corners["bl"])
    min_x = min([corners["tr"][0], corners["tl"][0], corners["br"][0], corners["bl"][0]])
    max_x = max([corners["tr"][0], corners["tl"][0], corners["br"][0], corners["bl"][0]])
    min_y = min([corners["tr"][1], corners["tl"][1], corners["br"][1], corners["bl"][1]])
    max_y = max([corners["tr"][1], corners["tl"][1], corners["br"][1], corners["bl"][1]])
    for point in points:
        if min_y <= Decimal(point[0][1]) <= max_y and min_x <= Decimal(point[0][0]) <= max_x:
            u1, u2 = symbols('u1 u2')
            eq1 = Eq(corners["tr"][0] + u1*float(v1[0]) + u2*float(v2[0]) - point[0][0], 0)
            eq2 = Eq(corners["tr"][1] + u1*float(v1[1]) + u2*float(v2[1]) - point[0][1], 0)
            sol = solve((eq1, eq2),(u1, u2))
            if 0 <= sol[u1] <= 1 and 0 <= sol[u2] <= 1:
                labels.add(point[1])
    return labels

In [ ]:
from rasterio import transform
def create_smaller_images(bands, bands_for_blue=None, path_name='output', file_name='tile', labeled_only=False):
    """Créée des images plus petites pouvant être mieux traitées par le réseau de neurones et les classes dans le bon dossier
    selon s'il existe des positions déjà labelée dans le dossier (ambigious si plusieurs, not_labeled si pas de coordonnées)"""
    images, images_for_blue, width, height, crs, transformFix, dtype, count, gcps = \
                                        sf.harmonize_s1bands(bands, bands_for_blue, image_pixel=IMAGE_PIXEL)
    if not os.path.isdir(path_name):
        os.mkdir(path_name)
    for _, directory in CORRESPONDANCES.items():
        if not os.path.isdir(path_name + "/" + directory):
            os.mkdir(path_name + "/" + directory)
    if not os.path.isdir(path_name + "/" + CORRESPONDANCES['-1'] + "/coffee_pepper"):
        os.mkdir(path_name + "/" + CORRESPONDANCES['-1'] + "/coffee_pepper") 
    if not os.path.isdir(path_name + "/" + CORRESPONDANCES['-1'] + "/coffee"):
        os.mkdir(path_name + "/" + CORRESPONDANCES['-1'] + "/coffee") 
    if not os.path.isdir(path_name + "/" + CORRESPONDANCES['-1'] + "/pepper"):
        os.mkdir(path_name + "/" + CORRESPONDANCES['-1'] + "/pepper") 
    if not os.path.isdir(path_name + "/" + CORRESPONDANCES['-1'] + "/other"):
        os.mkdir(path_name + "/" + CORRESPONDANCES['-1'] + "/other") 
    for x in range((height // IMAGE_PIXEL) + 1):
        for y in range((width // IMAGE_PIXEL) + 1):
            begin = transformFix * (y * IMAGE_PIXEL,x * IMAGE_PIXEL)
            transform = affine.Affine(10.0, 0, begin[0], 0.0, -10.0, begin[1])
            this_gcps = translate_gcps(gcps, IMAGE_PIXEL, x, y)
            #Trie selon s'il y a des points déjà répertorié dans le fichier.
            labels = get_labels(this_gcps)
            image_name = path_name + "/"
            if len(labels) == 0:
                if labeled_only:
                    continue
                if not os.path.exists(path_name + "/" + CORRESPONDANCES['0'] + "/" + str(x)):
                    os.mkdir(path_name + "/" + CORRESPONDANCES['0'] + "/" + str(x))
                image_name += CORRESPONDANCES['0'] + "/" + str(x) + "/"
            elif len(labels) > 1:
                if 2 in labels and (15 in labels or 25 in labels):
                    image_name += CORRESPONDANCES['-1'] + "/coffee_pepper/" 
                elif 2 in labels:
                    image_name += CORRESPONDANCES['-1'] + "/other/" 
                elif 15 in labels or 25 in labels:
                    image_name += CORRESPONDANCES['-1'] + "/pepper/" 
                else :
                    image_name += CORRESPONDANCES['-1'] + "/other/" 
            else:
                image_name += CORRESPONDANCES[str(next(iter(labels)))] + "/"
            image_name += file_name
            smaller_image = rasterio.open(image_name + '_' + str(x) + '_' + str(y) \
                                              + '.tiff', 'w', driver='Gtiff', 
                          width=IMAGE_PIXEL, height=IMAGE_PIXEL, count=count, crs=crs, 
                          gcps=this_gcps, transform=transform, 
                          dtype=dtype
                         )
            cpt = 1
            for i in range(len(images)):
                smaller_image.write(images[i][x][y], cpt)
                cpt += 1
                if bands_for_blue != None:
                    smaller_image.write(images_for_blue[i][x][y], cpt)
                    cpt += 1
                    smaller_image.write(make_blue(images[i][x][y], images_for_blue[i][x][y]), cpt)
                    cpt += 1
            smaller_image.close()
        print(str(x + 1) + "/" + str((height //IMAGE_PIXEL) + 1))
    print("Fichiers tiff créés.")

In [ ]:
create_smaller_images(bands, path_name=S1DIRECTORY, file_name=S1NAME, labeled_only=True)